In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import shutil

In [2]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json (2) ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json (2)

# Example: Replace username/dataset with the Kaggle dataset name
!kaggle datasets download -d yasserhessein/gender-dataset --unzip -p /content/dataset

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `cp kaggle.json (2) ~/.kaggle/'
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `chmod 600 ~/.kaggle/kaggle.json (2)'
Dataset URL: https://www.kaggle.com/datasets/yasserhessein/gender-dataset
License(s): unknown
 99% 1.31G/1.32G [00:18<00:00, 89.3MB/s]
100% 1.32G/1.32G [00:19<00:00, 74.2MB/s]


Now I shall sample any 1000 images out of test data set and the evaluate the accuracy of the mode

In [4]:
base_dataset_path = '/content/dataset/Dataset'


test_male_path = os.path.join(base_dataset_path, 'Test', 'Male')
test_female_path = os.path.join(base_dataset_path, 'Test', 'Female')

# Storing the sampled dataset in a new folder
sampled_test_path = '/content/sampled_test'
sampled_male_path = os.path.join(sampled_test_path, 'Male')
sampled_female_path = os.path.join(sampled_test_path, 'Female')

# Create directories for sampled data
os.makedirs(sampled_male_path, exist_ok=True)
os.makedirs(sampled_female_path, exist_ok=True)
male_images = os.listdir(test_male_path)
random.shuffle(male_images)  # Shuffle to randomize
for img in male_images[:1000]:  # Take the first 1000 after shuffling
    shutil.copy(os.path.join(test_male_path, img), os.path.join(sampled_male_path, img))

# Sample Female images
female_images = os.listdir(test_female_path)
random.shuffle(female_images)  # Shuffle to randomize
for img in female_images[:1000]:  # Take the first 1000 after shuffling
    shutil.copy(os.path.join(test_female_path, img), os.path.join(sampled_female_path, img))

print("Sampling complete!")
print(f"Male images sampled: {len(os.listdir(sampled_male_path))}")
print(f"Female images sampled: {len(os.listdir(sampled_female_path))}")

Sampling complete!
Male images sampled: 1000
Female images sampled: 1000


In [5]:
test_dir='/content/sampled_test'

Pre process to make it suitable for model

In [7]:
def detect_and_crop_face(image_path, target_size=(150, 150)):
    # Loading the image
    img = cv2.imread(image_path)
    # Converting into grayscale as HAAR CASCADE is designed to detect gray scale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detection of faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:  # If faces are found
        # Crop the first detected face
        x, y, w, h = faces[0]
        cropped_face = img[y:y+h, x:x+w]  # Crop the face region

        # Resize the cropped face to the target size
        cropped_face_resized = cv2.resize(cropped_face, target_size)

        return cropped_face_resized
    else:
        return None

**Making** **Labels**

In [8]:
test_male_path = '/content/sampled_test/Male'
test_female_path = '/content/sampled_test/Female'

def process_images(data_dir, label, target_folder):
    face_images = []
    labels = []

    # Going through all images in my directory
    for img_name in os.listdir(data_dir):
        img_path = os.path.join(data_dir, img_name)

        # Detect and crop face
        cropped_face = detect_and_crop_face(img_path)

        if cropped_face is not None:
            # Save the cropped face
            save_path = os.path.join(target_folder, str(label), img_name)
            cv2.imwrite(save_path, cropped_face)
            face_images.append(cropped_face)
            labels.append(label)

    return face_images, labels

# Create target directories for cropped images
os.makedirs('/content/cropped_faces/Male', exist_ok=True)
os.makedirs('/content/cropped_faces/Female', exist_ok=True)

# Process Male and Female images
male_faces, male_labels = process_images(test_male_path, 0, '/content/cropped_faces/Male')  # Label Male as 0
female_faces, female_labels = process_images(test_female_path, 1, '/content/cropped_faces/Female')  # Label Female as 1

# Combine the male and female data
faces = male_faces + female_faces
labels = male_labels + female_labels


faces = np.array(faces)
labels = np.array(labels)

# Normalize image data
faces = faces / 255.0

**IMPORTING THE BUILT MODEL AND CALCULTING ACCURACY ON** **TEST** **DATA**

In [13]:
import tensorflow as tf

# Load the saved model
from google.colab import drive
drive.mount('/content/drive')
model = tf.keras.models.load_model('/content/drive/MyDrive/gender_detection_model.keras')

# Evaluate the loaded model
test_loss, test_accuracy = model.evaluate(faces, labels)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
60/60 ━━━━━━━━━━━━━━━━━━━━ 29s 484ms/step - accuracy: 0.9300 - loss: 0.1796
Test Accuracy: 91.15%
